### Рюкзак
А именно *Multidimensional Multiple-Choice Quadratic Knapsack Problem* (MdMCQKP)

Пусть:
- N: количество предметов, которые можно положить в рюкзак
- M: размерность рюкзака, то есть количество различных ограничений в виде неравенств
- K: количество различных классов, на которые разбиваются предметы
- $x$: Бинарный вектор размера $N$

Тогда:
- profits (P): симметричная двухмерная матрица $N\times N$, где $P_{ij}$ это профит, если выбраны предметы $i$ и $j$ (для $i \not = j$: $P_{i, j}$ равен половине профита). Тогда $x^TPx$ это суммарный профит.
- groups (G): бинарная матрица размера $K \times N$, которая задает классы, для строчки $i$ будет выбран  один и только один предмет, среди которых в в соотвествующем столбце будет стоять $1$. То есть ограничение будет $\forall i \in \{1, \ldots K\}: \sum\limits_{j=1}^{N}G_{ij}x_j = 1$.
    Также можно записать как $Gx = \mathbb{1}_K$, где $\mathbb{1}_K$ - вектор из 1 размера K.
- capacity (c): вектор размера $M$, где $c_i$ равен вместимости рюкзака по измерению $i$.
- weights (W): матрица размера $M \times N$, где $W_{ij}$ равна весу(размеру) предмета $j$ для измерения рюкзака $i$.
То есть ограничение будет $\forall i \in \{1, \ldots M\}: \sum\limits_{j=1}^{N}W_{ij}x_j \le c_i$. Также можно записать это как $Wx \le c$, где неравенство имеется в виду по поокординатное.


### Переводим в ADMM задачу

Сначала введем еще одно обозначение.
Пусть новая переменная $u \in \mathbb{R}_{\ge 0}^M$ и мы заменим $Wx < c$ на равенство $Wx + u = c$. Но это тоже самое, что $Wx + u - c = 0$, а это эквивалентно $\|Wx + u - c\|_2^2 \le 0$

В IV секции в статье про ADMM у нас получились обозначения:
- $q(x) = -x^TPx$ - является квадратичным
- $\mathcal{X}$ - это множество из всех бинарных векторов размерности N
- $\mathcal{U} = \mathbb{R}_{\ge 0}^M, u = u$ - множество будет выпуклым
- $\varphi(u) = 0$, оно у нас не используется
- $G = G$, $b = \mathbb{1}_K$ тут у нас обозначения совпали
- $g(x) = 0$, оно у нас не используется
- $\ell(x, u)$ заменится на $\|Wx + u - c\|_2^2$ (x и u в качестве себя) - это функция должна быть совместно выпуклой, сейчас я это не буду доказывать

Наша текущая задача 

\begin{equation}
\min_{x\in\mathcal{X}, u \in \mathbb{R}_{\ge 0}^M \subset \mathbb{R}^M} -x^TPx
\end{equation}
\begin{equation}
\text{subject to: }Gx=\mathbb{1}_K, \| Wx + u - c \|_2^2 \le 0
\end{equation}



Сделаем следующий шаг согласно статье, когда мы вводим новую переменную $z$ и ослабляем равенство $Gx=\mathbb{1}_K$. Пусть $\alpha > 0$ - какое очень большое число

\begin{equation}
\min_{x\in\mathcal{X}, z \in \mathbb{R}^N, u \in \mathbb{R}_{\ge 0}^M} -x^TPx + \dfrac{\alpha}{2}\|Gx - \mathbb{1}_K\|_2^2
\end{equation}
\begin{equation}
\text{subject to: }\| Wz + u - c \|_2^2 \le 0, x = z
\end{equation}


Теперь мы назовем вектор $\overline{x}$ размера $N + M$, где первые $N$ чисел образуют вектор $z$, а последние $M$ образуют вектор $u$, то есть мы вертикально их разместили, $z$ над $u$ \n

Тогда:
- $f_0(x) := -x^TPx + \dfrac{\alpha}{2}\|Gx - \mathbb{1}_K\|^2_2$
- $\overline{X} := \left\{(z\in\mathbb{R}^N, u\in\mathbb{R}_{\ge0}^M)\Big| \|Wz + u - c\|_2^2 \le 0\right\}$
- $\iota_X(x) := 0$ если $x\in X$, иначе равна $+\infty$, это функция индикатор, которая снизу полунепрерывная
- В приведенной статье $f_1(\overline{x}) := \iota_{\overline{X}}(\overline{x})$
- **В данной статье** зададим функцию как $f_1(\overline{x}) := \dfrac{\beta}{2}\|Wz + u - c\|_2^2$, где $z, u$ такие, что $\overline{x} = [z^T; u^T]^T$

Тогда задача станет (2-ADMM-H):
\begin{equation}
\min_{x\in\mathcal{X}, \overline{x}\in\mathbb{R}^{N+M}} f_0(x) + f_1(\overline{x})
\end{equation}
\begin{equation}
\text{subject to: }A_0x + A_1\overline{x} = 0
\end{equation}

где:
- $A_0 = E_N$ - единичная матрица размера $N\times N$
- $A_1 = [-E_N; 0_{N\times M}]$, получится матрица размера $N \times (N + M)$

Это мы свели задачу к 2-блочному ADMM задачу

Теоретически нам достаточно 2-блочного ADMM, так как у нас есть гладкость $f_1(\overline{x})$, но мы можем свести к 3-блочному ADMM, потому что на практике он быстрее сходится.

Тогда задача станет (3-ADMM-H):

\begin{equation}
\min_{x\in\mathcal{X}, \overline{x}\in\mathbb{R}^{N+M}, y\in\mathbb{R}^{N}} f_0(x) + f_1(\overline{x}) + \dfrac{\gamma}{2}\|y\|^2_2
\end{equation}
\begin{equation}
\text{subject to: }A_0x + A_1\overline{x} = y
\end{equation}

#### Выпуклость $f_1$

Перед тем, как начать расписывать подробно алгоритм, докажем, что $f_1$ является выпуклой функией от $\overline{x}$. (Понятно, что умножение на константу не влияет на выпуклость). Пусть $\overline{x} = [z^T; u^T]^T$

$$f_1(\overline{x}) = \|Wz + u - c\|_2^2$$

Перепишем функцию по другому

$$f_1(\overline{x}) = \|[W, E_M]\overline{x} - c\|_2^2$$

Но заметим, что $[W, E_M]\overline{x} - c$ является аффиной картой между $\mathbb{R}^{N+M}$ и $\mathbb{R}^{M}$. Также мы знаем $\|\cdot\|_2^2$ является выпуклой функцией. Но тогда просто скажем, что свойство выпуклости сохраняется даже при аффинных карт. Вот и все доказательство

#### Алгоритм 3-ADMM-H

Распишем подробей шаги для трехблочного алгоритма

Пусть мы как-то инициилизировали начальные переменные $x_0, \overline{x}_0, \lambda_0$, также зададим константы $epochs$, $\varrho$, $\alpha$, $\beta$, $\gamma$, $\mu$, $\varepsilon$

Тогда мы $epochs$ раз будем повторять шаги, пусть переменная $t$ говорит номер эпохи начиная с $1$ (также же проверяется, что $\|A_0x_t + A_1\overline{x}_k - y_k\|_2 < \varepsilon$, иначе останавливаемся):

1. **QUBO блок**
   
   В этом блоке будем обновлять $x_t$, решая QUBO задачу.
   $$x_t = \underset{x\in\{0,1\}^N}{\arg\min} -x^TPx+\dfrac{\alpha}{2}\|Gx - \mathbb{1}_K\|_2^2 + \lambda_{t-1}^TA_0x + \dfrac{\varrho}{2}\|A_0x + A_1\overline{x}_{t-1} - y_{t-1}\|^2_2$$

   Подставим значия $A_0=E_N, A_1=[-E_N, 0_{N\times M}]$, так же пусть $\overline{x}_t = [z_t^T, u_t^T]^T$

   $$x_t = \underset{x\in\{0,1\}^N}{\arg\min} -x^TPx+\dfrac{\alpha}{2}\|Gx - \mathbb{1}_K\|_2^2 + \lambda_{t-1}^Tx + \dfrac{\varrho}{2}\|x - z_{t-1}-y_{t-1}\|^2_2$$

    Распишем $\|Eq\|_2^2$ как $Eq^TEq$ для векторов

   $$x_t = \underset{x\in\{0,1\}^N}{\arg\min} -x^TPx+\dfrac{\alpha}{2}(Gx - \mathbb{1}_K)^T(Gx - \mathbb{1}_K) + \lambda_{t-1}^Tx + \dfrac{\varrho}{2}(x - z_{t-1}-y_{t-1})^T(x - z_{t-1}-y_{t-1})$$

    Раскроем скобки, уберем константы

   $$x_t = \underset{x\in\{0,1\}^N}{\arg\min} -x^TPx+\dfrac{\alpha}{2}(x^TG^TGx - \mathbb{1}_K^TGx - x^TG^T\mathbb{1}_K) + \lambda_{t-1}^Tx + \dfrac{\varrho}{2}(x^Tx - (z_{t-1}+y_{y_{t-1}})^Tx - x^T(z_{t-1}+y_{t-1}))$$

    Приводим в красивый вид (замечу, что $b^Tx = x^Tb$ при условии, что $b$ - вектор, так как $b^Tx$ - скаляр)

   $$x_t = \underset{x\in\{0,1\}^N}{\arg\min}\ 
   x^T\left(-P+\frac{\alpha}{2}G^TG+\frac{\varrho}{2}E_N\right)x - 
   \left(\alpha G^T\mathbb{1}_K + \varrho (z_{t-1}+y_{t-1}) - \lambda_{t-1}\right)^Tx$$

   Чтобы сделать задачу $x^TAx + b^Tx$ чисто $x^TQx$ при условии, что $x$ - бинарный вектор, то надо просто на главную диагональ $A$ добавить вектор $b$, так как $x_{[i]}^2 = x_{[i]}$, где $x_{[i]}$ - $i$-ая координата $x$
3. **Convex block**

   **Первый способ**

   Что нам нужно

   $$\overline{x}_k = \underset{z\in\mathbb{R}^{N},u\in\mathbb{R}_{\ge0}^M, \overline{x}=[z^T;u^T]^T}{\arg\min}\ \dfrac{\beta}{2}\|[Wz + u - c \|_2^2 + \lambda_{t-1}^Tz + \dfrac{\varrho}{2}\|x_t+z-y_{t-1}\|_2^2$$

   Но на самом деле же нам надо сделать $Wz + u = c$ при $u \in \mathbb{R}^M$ и минимизировать остальную часть

   $$\overline{x}_k = \underset{z\in\mathbb{R}^{N},\overline{x}=[z^T;u^T]^T}{\arg\min}\ \dfrac{\varrho}{2}z^Tz + \lambda_{t-1}^Tz + \varrho(x_t-y_{t-1})^Tz$$

   $$\text{subject to: } Wz + u = c,\ u \in \mathbb{R}^M_{\ge0}$$

   Такое принимает решатель

4. **Convex+quadratic block**

   В этом блоке будем обновлять $y_k$

    $$y_k = \underset{y\in\mathbb{R}^N}{\arg\min}\ \dfrac{\gamma}{2}\|y\|_2^2 - \lambda_{t-1}^Ty+\dfrac{\varrho}{2}\|x_t+A_1\overline{x}_t-y\|_2^2$$

   Заменим $\|\cdot\|_2^2$

   $$y_k = \underset{y\in\mathbb{R}^N}{\arg\min}\ \dfrac{\gamma}{2}y^Ty - \lambda_{t-1}^Ty+\dfrac{\varrho}{2}(x_t+A_1\overline{x}_t-y)^T(x_t+A_1\overline{x}_t-y)$$

   Раскроем и сократим

   $$y_k = \underset{y\in\mathbb{R}^N}{\arg\min}\ \dfrac{\gamma}{2}y^Ty - \lambda_{t-1}^Ty+\dfrac{\varrho}{2}(-(x_t+A_1\overline{x}_t)^Ty - y^T(x_t+A_1\overline{x}_t) - y^Ty)$$

   Приведем в красивый вид

   $$y_k = \underset{y\in\mathbb{R}^N}{\arg\min}\ \dfrac{\gamma+\varrho}{2}y^Ty - \lambda_{t-1}^Ty - \varrho(x_t+A_1\overline{x}_t)^Ty$$

   Попробуем решить самим, без решателя. Пусть мы обозначим $i$ координату вектора $v$ как $v[i]$

    $$y_k[i] = \underset{y\in\mathbb{R}}{\arg\min}\ \dfrac{\gamma+\varrho}{2}y[i]^2 - \lambda_{t-1}[i]y[i] - \varrho(x_t+[-E_N;0_{N\times M}]\overline{x}_t)[i]y[i]$$

   Тоже самое

   $$y_k[i] = \underset{y\in\mathbb{R}}{\arg\min}\ \dfrac{\gamma+\varrho}{2}y[i]^2 - \lambda_{t-1}[i]y[i] - \varrho(x_t[i]-\overline{x}_t[i])y[i]$$

   Но заметим, что тут частная производная получается очевидная и градиент соотвественно

   $$\nabla y = (\gamma + \varrho)y - \lambda_{t-1}-\varrho(x_t - [E_N;0_{N \times M}]\overline{x}_t)$$.

   Заметим также, что вторые частные производные равны все $(\gamma + \varrho)$ и они тогда положительный, а значит минимум будет не бесконечным.

   Минимальное значение должно быть при градиенте равным нулю, а значит

   $$ y_k = \dfrac{\lambda_{t-1} + \varrho(x_t - [E_N;0_{N \times M}]\overline{x}_t)}{\gamma + \varrho}$$

5. Обновляем $\lambda_t$
    $$\lambda_t = \lambda_{t-1} + \varrho(A_0x_t + A_1\overline{x}_t - y_t)$$
6. Считаем специальную метрику, по которой будем выбирать лучшее решение, где $l$ - какая-то функция ошибки от $x_t$ и $\overline{x}_t$

    $$\eta_t = -x_t^TPx_t + \mu\max(l(x_t, \overline{x}_t), 0)$$